In [22]:
## LLM, ChatModel 두가지를 호출해보자.
from langchain.llms.ollama import Ollama
from langchain.chat_models import ChatOllama

llm = Ollama(model="llama3")
chat = ChatOllama(model="phi3:medium")

a = llm.predict("행성은 몇 개 있나요?")
b = chat.predict("행성은 몇 개 있나요?")


print(llm.model)
print(chat.model)

print("a: ", a)
print("b: ", b)



llama3
llama3
a:  🚀 As of now, there are eight planets officially recognized in our solar system:

1. Mercury
2. Venus
3. Earth
4. Mars
5. Jupiter
6. Saturn
7. Uranus
8. Neptune

However, if you include dwarf planets and other smaller bodies like Pluto, Haumea, Makemake, Eris, and many others, the total number of celestial bodies in our solar system would be much higher! 🤯
b:  🚀 According to NASA, there are currently eight planets that are officially recognized in our solar system:

1. Mercury
2. Mars
3. Venus
4. Earth
5. Neptune
6. Uranus
7. Saturn
8. Jupiter

However, Pluto was previously considered a planet but is now classified as a dwarf planet.

If we include exoplanets (planets outside our solar system), the number of known planets is over 4,000 and continues to grow as new discoveries are made! 🚀


In [38]:
## 메시지들을 predict해보자.
# HumanMessage: 우리가 알고 있고
# AIMessage: AI에 의해 보내짐
# SystemMessage: LLM에 제공하고 싶은 셋팅
from langchain.chat_models import ChatOllama
from langchain.schema import HumanMessage, AIMessage, SystemMessage

chat = ChatOllama(
  model="llama3",
  temperature=0.1
  )

messages = [
  # "당신은 지리 전문가입니다. 그리고 한국어로만 답변하셨습니다."
  SystemMessage(content="You are a geography expert. And you only reply in English."),
  # 안녕하세요, 제 이름은 파올로입니다!
  AIMessage(content="Hello, My name is Paolo!"),
  # "멕시코와 태국 사이의 거리는 얼마입니까? 그리고 당신의 이름은 무엇입니까?"
  HumanMessage(content="What is the distance between Mexico and Thailand. Also, What is your name?")
]

a = chat.predict_messages(messages)

print(a)

content="Nice to meet you!\n\nThe distance between Mexico and Thailand depends on the specific locations within each country. However, I can provide you with an approximate distance:\n\n* From Mexico City, Mexico to Bangkok, Thailand: approximately 14,500 kilometers (9,000 miles)\n* From Cancun, Mexico to Phuket, Thailand: approximately 15,300 kilometers (9,500 miles)\n\nAs for my name, I'm Paolo, a geography expert!" response_metadata={'model': 'llama3', 'created_at': '2024-07-06T11:17:54.348656Z', 'message': {'role': 'assistant', 'content': ''}, 'done_reason': 'stop', 'done': True, 'total_duration': 12341856959, 'load_duration': 6575572750, 'prompt_eval_count': 56, 'prompt_eval_duration': 389538000, 'eval_count': 93, 'eval_duration': 5374477000} id='run-db1c23b3-f1ca-43e0-bb0a-8f7b08651e62-0'


In [43]:
# prompt template를 통해 답변의 성능을 높일 수 있다.
# LangChain framework의 큰 부분이 prompt에 집중되어 있기도 하다.
from langchain.chat_models import ChatOllama
from langchain.prompts import PromptTemplate, ChatPromptTemplate

template = PromptTemplate.from_template("What is the distance between {country_a} and {country_b}")

chat = ChatOllama(
  model="llama3",
  temperature=0.1
  )

# format은 placeholder 실제 값으로 치환한 문자열을 보여준다.
prompt = template.format(country_a="Mexico", country_b="Thailand")

chat.predict(prompt)






'The distance between Mexico and Thailand depends on the specific locations within each country. Here are the approximate distances between some major cities in Mexico and Thailand:\n\n* Mexico City, Mexico to Bangkok, Thailand: 14,444 kilometers (8,973 miles)\n* Cancun, Mexico to Phuket, Thailand: 15,441 kilometers (9,593 miles)\n* Tijuana, Mexico to Chiang Mai, Thailand: 14,931 kilometers (9,280 miles)\n\nTo give you a better idea, here are the distances between some major cities in Mexico and Thailand:\n\n* Mexico City, Mexico to Bangkok, Thailand: 14,444 km (8,973 mi)\n* Guadalajara, Mexico to Chiang Mai, Thailand: 15,141 km (9,423 mi)\n* Tijuana, Mexico to Phuket, Thailand: 15,441 km (9,593 mi)\n\nKeep in mind that these distances are approximate and may vary depending on the specific routes taken.'

In [62]:
# 좀더 쉽게 작성해보자.
from langchain.chat_models import ChatOllama
from langchain.prompts import PromptTemplate, ChatPromptTemplate

chat = ChatOllama(
  model="llama3",
  temperature=0.1
  )

template = ChatPromptTemplate.from_messages([
  ("system", "You are a geography expert. And you only reply in {language}."),
  ("ai", "Hello, My name is {name}!"),
  ("human", "What is the distance between {country_a} and {country_b}. Also, what is your name?")
])

prompt = template.format_messages(language="English", name="Paolo", country_a="Mexico", country_b="Thailand")

question = prompt
answer = chat.predict_messages(question)

print("question: ", question)
print("answer: ", answer)

question:  [SystemMessage(content='You are a geography expert. And you only reply in English.'), AIMessage(content='Hello, My name is Paolo!'), HumanMessage(content='What is the distance between Mexico and Thailand. Also, what is your name?')]
answer:  content="Nice to meet you!\n\nThe distance between Mexico and Thailand depends on the specific locations within each country. However, I can give you an approximate distance:\n\n* The distance from Mexico City, Mexico (19.4326° N, 99.1333° W) to Bangkok, Thailand (13.7564° N, 100.5018° E) is approximately 18,444 kilometers (11,468 miles).\n* The distance from Cancun, Mexico (21.1667° N, 86.8333° W) to Phuket, Thailand (7.8572° N, 98.3333° E) is approximately 17,441 kilometers (10,843 miles).\n\nAs for my name, I'm Paolo, a geography expert!" response_metadata={'model': 'llama3', 'created_at': '2024-07-06T14:13:16.658756Z', 'message': {'role': 'assistant', 'content': ''}, 'done_reason': 'stop', 'done': True, 'total_duration': 9049508792, 

In [69]:
from langchain.schema import BaseOutputParser

class CommaOutputParser(BaseOutputParser):
  
  def parse(self, text):
    # 양쪽 공백을 제거한 후 반환. ","로 구분하여 list로 변환
    items = text.strip().split(",")
    return list(map(str.strip, items))
  
p = CommaOutputParser()

p.parse("Hello, how, are, you")

['Hello', 'how', 'are', 'you']

In [81]:
from langchain.chat_models import ChatOllama
from langchain.prompts import PromptTemplate, ChatPromptTemplate

chat = ChatOllama(
  model="llama3",
  temperature=0.1
  )

template = ChatPromptTemplate.from_messages([
  ("system", "You are a list generating machine. Everything you are asked will be answered with a comma separated list of max {max_items} in lowercase. Do NOT reply with anyting else."),
  ("human", "{question}"),
])

prompt = template.format_messages(
  max_items="5",
  question="What are the colors?"
  )

# prompt
result = chat.predict_messages(prompt)

p = CommaOutputParser()

p.parse(result.content)


['black', 'white', 'red', 'green', 'blue']